<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema2_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento de datos para su análisis y visualización

En este notebook vamos a importar de [EUROSTAT](https://ec.europa.eu/eurostat/web/main/data/database) datos de PIB y paro para los países de la UE y a procesarlos para poder, posteriormente, visualizarlos y analizarlos.

La gran ventaja de importar y procesar los datos en Python es que, al automatizar el proceso, resulta mucho más rápido volver a realizar la misma tarea. Además, tener el código escrito nos ayuda a identificar y a corregir errores, los cuales son muy difíciles de detectar si realizamos la manipulación de datos en Excel.

*Daniel Oto-Peralías*

### Importamos las librerías que necesitamos

In [ ]:
# IMPORTAMOS LIBRERIAS
import pandas as pd
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 30)
import matplotlib.pyplot as plt

### PIB en volumen

In [ ]:
# PIB en volúmen (Gross domestic product, volumes (teina011))
link_pib="https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/teina011?format=TSV&compressed=true"
pib = pd.read_csv(link_pib, compression='gzip', header=0, sep='\t')
pib # En el excel de EUROSTAT podemos comprobar que son tablas equivalentes.

Vamos a hacer el siguiente procesamiento a los datos:
1. Dividimos la columna ```freq,na_item,unit,geo\TIME_PERIOD``` en cuatro columnas.
2. Seleccionamos solo la unidad "PCH_Q1_SCA" que significa "Percentage change, Annual variation rate, Seasonally and Calendar Adjusted.
3. Borramos los valores "p" y los ":".
4. Transponemos la tabla.
5. Usamos los nombres de países como nombre de las columnas.
6. Convertimos las columnas en formato numérico.
7. Establecemos el index de la tabla como fecha.
8. Guardamos los datos en un fichero csv.


In [ ]:
# 1. Dividimos la columna en tres.
# Hay que separar por la coma
pib[['freq','na_item','unit','geo']]=pib[r'freq,na_item,unit,geo\TIME_PERIOD'].str.split(",", expand=True)
pib.drop(columns=r'freq,na_item,unit,geo\TIME_PERIOD',inplace=True)
pib

In [ ]:
# 2. Seleccionamos solo la variable PCH_Q4_SCA
pib=pib.loc[pib['unit']=="PCH_Q4_SCA"]
pib

In [ ]:
# 3. Borramos los valores p y los :
for col in pib.columns:
  pib[col]=pib[col].replace("p","",regex=True) # Dejamos espacio adelante porque dicho espacio existe en los datos.
  pib[col]=pib[col].replace(":","",regex=True)
  pib[col]=pib[col].replace(" ","",regex=True) # por si hay algún espacio
pib

In [ ]:
# 4. Transponemos la tabla
pib=pib.transpose()
pib

In [ ]:
# 5. Usamos la fila de nombres de países como nombres de las columnas
nombre_col = pib.iloc[-1] # Los nombres de países están en la última fila
pib = pib.iloc[:-4] # aplicamos un filtrado para quitar las 4 últimas filas
pib.columns = nombre_col # Establecemos el nuevo nombre de las columnas
pib

In [ ]:
# 6. Convertimos las columnas en formato numérico.
for col in pib.columns:
  pib[col]=pd.to_numeric(pib[col], errors='coerce')
pib

In [ ]:
#7. Establecemos el index de la tabla como fecha
pib.index=pib.index.str.replace(" ","",regex=True)
pib.index=pd.to_datetime(pib.index)
pib

In [ ]:
#8. Guardamos los datos en un fichero csv.
pib.to_csv("pib_volumen_EUROSTAT.csv")

Nótese que el archivo "pib_volumen_EUROSTAT.csv" se ha guardado en la carpeta de archivos de Google Colab. Por tanto, tenemos que descargarla en nuestro ordenador.

### Tasa de desempleo

Repetimos los mismos pasos que anteriormente para el PIB, pero adaptando el código a las particularidades de la tabla de datos del paro.

In [ ]:
# Tasa de desempleo (Total unemployment rate (tps00203))
link_des="https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/une_rt_q?format=TSV&compressed=true"
tp = pd.read_csv(link_des, compression='gzip', header=0, sep='\t')
tp

In [ ]:
# 1. Dividimos la columna en seis (hay que separar por la coma)
columna=r'freq,s_adj,age,unit,sex,geo\TIME_PERIOD'
tp[['freq','s_adj','age','unit','sex','geo']]=tp[columna].str.split(",", expand=True)
tp.drop(columns=columna,inplace=True)
tp

In [ ]:
# Exploramos un poco los datos
tp.unit.value_counts()


In [ ]:
# 2. Seleccionamos SA, Y15-74, PC_ACT, T
tp=tp.loc[(tp['s_adj']=="SA") &
          (tp['age']=="Y15-74") &
          (tp['unit']=="PC_ACT") &
          (tp['sex']=="T")]
tp

In [ ]:
# 3. Borramos los valores p y los :
for col in tp.columns:
  for c in ["p","d","u","b",":"," "]:
    tp[col]=tp[col].replace(c,"",regex=True)
tp

In [ ]:
# 4. Transponemos la tabla
tp=tp.transpose()
tp

In [ ]:
# 5. Usamos la fila de nombres de países como nombres de las columnas
nombre_col = tp.iloc[-1] # Los nombres de países están en la última fila
tp = tp.iloc[:-6] # aplicamos un filtrado para quitar las seis últimas filas
tp.columns = nombre_col # Establecemos el nuevo nombre de las columnas

# 6. Convertimos las columnas en formato numérico.
for col in tp.columns:
  tp[col]=pd.to_numeric(tp[col], errors='coerce')

#7. Establecemos el index de la tabla como fecha
tp.index=tp.index.str.replace(" ","",regex=True)
tp.index=pd.to_datetime(tp.index)
tp.sort_index(inplace=True)

#8. Guardamos los datos en un fichero csv.
tp.to_csv("Tasa_paro_EUROSTAT.csv")
tp